## Load Files and Vectorize 

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import textstat
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

In [4]:
with open(r"data/df_corpus2.pkl", "rb") as input_file:
    df_corpus2 = pickle.load(input_file)

In [3]:
with open(r"data/df_best_category.pkl", "rb") as input_file:
    df_best_category = pickle.load(input_file)

In [5]:
with open (r"data/dictionary.pkl", "rb") as input_file: 
    dictionary = pickle.load(input_file)

In [29]:
with open (r"data/lda_model_1.pkl", "rb") as input_file:
    lda_model_1 = pickle.load(input_file)

In [41]:
with open(r"data/df_best_category.pkl", 'rb') as input_file:
      df_best_category = pickle.load(input_file)

In [42]:
df_best_category.head()

,category,article_index
0,2,0
1,5,1
2,5,2
3,4,3
4,6,4


## Third Model 

In [ ]:
import textstat
import operator
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
from operator import itemgetter

In [78]:
lemmatizer = WordNetLemmatizer()

def lemmatize_stemming(text, lemmatizer=lemmatizer):
    return lemmatizer.lemmatize(text, pos='v')

In [79]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [75]:
def give_harder_level(text, vectorizer=vectorizer):
    """
    Takes in text and returns a harder level read
    """
    unseen_document = str(text)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    text_topics = lda_model_1[bow_vector]
    topic_group = max(text_topics,key=itemgetter(1))[0]
    
    category_df = df_best_category.loc[df_best_category['category'] == topic_group] 
    
    ##find all the articles that are easier than this article 
#     all_harder_text = []
#     input_score = textstat.flesch_kincaid_grade(text)
#     for i in range(len(df_corpus2['score'])):
#         if df_corpus2['score'][i] > input_score:
#             all_harder_text.append(df_corpus2['content'][i])
    
    all_harder_text = []
    
    for i in category_df['article_index']: 
        temp = df_corpus2['content'][i]
        all_harder_text.append(temp)
        
    vec = TfidfVectorizer(stop_words='english', max_features=2000)
    vec = vec.fit(all_harder_text)
    corpus2_vectors = vec.transform(all_harder_text).toarray()
    user_doc = text
    user_doc_vector = vec.transform([user_doc]).toarray() 
    #user_doc_vector_short = np.argsort(user_doc_vector)
    
    top_features = np.argsort(user_doc_vector)[:, -20:][::-1]
    
    distances = cdist(user_doc_vector[:, top_features],
                  corpus2_vectors[:, top_features],
                  metric='cosine')[0]
    ranking = np.argsort(distances)
    top = ranking[0]
    best_match = df_corpus2['content'][top]
        
    print(distances[top])
    return (best_match)

In [76]:
sample = "Wine is an alcoholic drink made from fermented grapes.[1] Yeast consumes the sugar in the grapes and converts it to ethanol, carbon dioxide, and heat. Different varieties of grapes and strains of yeasts produce different styles of wine. These variations result from the complex interactions between the biochemical development of the grape, the reactions involved in fermentation, the terroir, and the production process. Many countries enact legal appellations intended to define styles and qualities of wine. These typically restrict the geographical origin and permitted varieties of grapes, as well as other aspects of wine production. Wines not made from grapes include rice wine and fruit wines such as plum, cherry, pomegranate, currant and elderberry."

In [77]:
give_harder_level(sample)

ValueError: XA must be a 2-dimensional array.

> /Users/sherzyang/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py(2698)cdist()
   2696 
   2697     if len(s) != 2:
-> 2698         raise ValueError('XA must be a 2-dimensional array.')
   2699     if len(sB) != 2:
   2700         raise ValueError('XB must be a 2-dimensional array.')

ipdb> q


# Experiment

In [82]:
df_corpus2.head()

,content,score
0,"Nature Reviews Nephrology (before 2009, Nature...",20.6
1,Food blogging represents a complex interweavin...,13.6
2,The International Motorcycle Shows are a serie...,13.3
3,"Education in Ancient Greece was vastly ""democr...",11.3
4,Rural economics is the study of rural economie...,12.5


In [125]:
n_features = 2000

In [126]:
vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')

In [127]:
vectorizer.fit(df_corpus2['content'])
corpus_vectors = vectorizer.transform(df_corpus2['content'])

In [128]:
with open('data/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
    
with open('data/corpus_vectors.pkl', 'wb') as f:
    pickle.dump(corpus_vectors, f)

In [129]:
def load_vectorizer(pickle_file='data/vectorizer.pkl'):
    """Loads the trained TF/IDF vectorizer."""
    with open(pickle_file, 'rb') as f:
        return pickle.load(f)
    
def load_corpus_vectors(pickle_file='data/corpus_vectors.pkl'):
    """Loads the corpus vectors."""
    with open(pickle_file, 'rb') as f:
        return pickle.load(f)

In [130]:
vec = load_vectorizer()
corpus_vectors = load_corpus_vectors().toarray()

In [195]:
sample

'Wine is an alcoholic drink made from fermented grapes.[1] Yeast consumes the sugar in the grapes and converts it to ethanol, carbon dioxide, and heat. Different varieties of grapes and strains of yeasts produce different styles of wine. These variations result from the complex interactions between the biochemical development of the grape, the reactions involved in fermentation, the terroir, and the production process. Many countries enact legal appellations intended to define styles and qualities of wine. These typically restrict the geographical origin and permitted varieties of grapes, as well as other aspects of wine production. Wines not made from grapes include rice wine and fruit wines such as plum, cherry, pomegranate, currant and elderberry.'

In [189]:
sample_vector = vec.transform([sample]).toarray()

In [190]:
sample_vector

array([[0., 0., 0., ..., 0., 0., 0.]])

In [196]:
feature_ranking = np.argsort(sample_vector[0])[::-1]

In [197]:
feature_ranking[:20]

array([1973, 1783, 1448,  341, 1586, 1791,  585,  624,  988,  548,  986,
       1081, 1301, 1443,  229,  998,  443, 1894,  493, 1440])

In [198]:
def get_vocab_arr(vec):
    n_features = len(vec.vocabulary_)
    vocab_arr = np.empty(n_features, dtype=object)
    for word, idx in vec.vocabulary_.items():
        vocab_arr[idx] = word
    return vocab_arr

In [199]:
vocab_arr = get_vocab_arr(vec)
vocab_arr[feature_ranking[:20]]

array(['wine', 'styles', 'production', 'carbon', 'rice', 'sugar',
       'different', 'drink', 'interactions', 'define', 'intended',
       'legal', 'origin', 'produce', 'aspects', 'involved', 'complex',
       'typically', 'countries', 'process'], dtype=object)

In [200]:

def get_top_k_vector(vector, feature_ranking, k=20):
    """Return the top k vector according to feature_ranking."""
    return vector[:, feature_ranking[:k]]
    

In [201]:
sample_vector.shape, corpus_vectors.shape

((1, 2000), (14216, 2000))

In [202]:
(
    get_top_k_vector(sample_vector, feature_ranking).shape,
    get_top_k_vector(corpus_vectors, feature_ranking).shape
)

((1, 20), (14216, 20))

In [203]:
distances = cdist(
    get_top_k_vector(sample_vector, feature_ranking),
    get_top_k_vector(corpus_vectors, feature_ranking),
)

In [204]:
get_top_k_vector(sample_vector, feature_ranking)

array([[0.7814979 , 0.27212003, 0.19489432, 0.16540916, 0.15650855,
        0.15059081, 0.14286366, 0.13943489, 0.13913053, 0.13056839,
        0.11549873, 0.11307387, 0.11024919, 0.10755569, 0.10538866,
        0.10182389, 0.10157666, 0.09993577, 0.09456803, 0.08846925]])

In [205]:
nearest_article_idxs = np.argsort(distances)

In [206]:
nearest_article_idxs.shape

(1, 14216)

In [207]:
nearest_article_idxs[0, 0]

8978

In [208]:
distances[0]

array([0.99093937, 0.97773468, 0.99093937, ..., 0.98314412, 0.97880632,
       1.08141708])

In [209]:
nearest_articles = df_corpus2.loc[nearest_article_idxs[0], :]

In [210]:
nearest_articles

,content,score
8978,There is a long history of alcoholic drinks in...,13.4
12149,A cocktail is a mixed drink typically made wit...,21.4
12189,The earliest archaeological evidence of wine p...,13.1
11705,The world's religions have had differing relat...,17.9
9839,"A spritzer is a tall, chilled drink, usually m...",16.0
5693,This is a list of alcoholic drinks. An alcohol...,35.2
12498,The Declaration to Protect Wine Place and Orig...,23.9
3324,The influence of wine in ancient Greece helped...,13.5
5602,Food & Wine is a monthly magazine published by...,14.9
660,Fermentation in food processing is the process...,19.7
